In [1]:
import numpy as np
import gym
from gym.wrappers import Monitor
import time
import glfw
from tqdm import tqdm
import json
import codecs
import argparse


In [2]:
xml_path = "/home/ghost-083/Research/1_Transfer_RL/Task_similarity/env_mods/assets/ant/ant_5.xml"
env = gym.make('Ant-v3', xml_file=xml_path)
curr_obs = env.reset(seed=908778)   # make sure seed is same for all experiments
terminated = False
for k in range(2):
    action = env.action_space.sample()
    next_obs, reward, terminated, info = env.step(action)
    curr_obs = next_obs                    
    time.sleep(0.01)



In [ ]:
0